# Mapping a Title to a Single SOC Code
***
The purpose of this walkthrough is to demonstrate one way to use Emsi's Titles API in combination with the Job Postings API to create a mapping of Emsi titles to its most common SOC code.

In [1]:
# import the libraries
from EmsiApiPy import TitlesConnection, UnitedStatesPostingsConnection

# for pretty printing
import json

titles_conn = TitlesConnection()
jpa_conn = UnitedStatesPostingsConnection()

In [2]:
# first, we'll get a list of all the titles in the emsi library and their default mapping to a SOC
data = titles_conn.get_list_all_titles(fields = ['id', 'name', 'mapping'])

# we'll print just the first two records
print(json.dumps(data[:2], indent = 2))

[
  {
    "id": "ET4A446A1A5F6142AD",
    "name": ".NET Application Architect",
    "mapping": {
      "skills": [
        {
          "id": "KS1200B62W5ZF38RJ7TD",
          "name": ".NET Framework"
        }
      ],
      "socs": [
        {
          "id": "15-1256",
          "name": "Software Developers and Software Quality Assurance Analysts and Testers"
        }
      ]
    }
  },
  {
    "id": "ET2636514E20FBBF10",
    "name": ".NET Application Developer",
    "mapping": {
      "skills": [
        {
          "id": "KS1200B62W5ZF38RJ7TD",
          "name": ".NET Framework"
        }
      ],
      "socs": [
        {
          "id": "15-1256",
          "name": "Software Developers and Software Quality Assurance Analysts and Testers"
        }
      ]
    }
  }
]


In [6]:
# As we can see, the first two titles map simply to only one occupation code.
# let's find an example that has more than one
multiples = [record for record in data if len(record['mapping']['socs']) > 1 and len(record['mapping']['skills']) > 0]
print(len(multiples))

608


In [7]:
# So there are about 6k titles in Emsi's library that have more than one SOC mapped to them
# here's the first one on the list
print(json.dumps(multiples[0], indent = 2))

{
  "id": "ET0E8AC162972F4096",
  "name": "ABA Behavioral Therapist",
  "mapping": {
    "skills": [
      {
        "id": "KS120GS6SC8WL88NJD9Y",
        "name": "Applied Behavior Analysis"
      }
    ],
    "socs": [
      {
        "id": "21-1013",
        "name": "Marriage and Family Therapists"
      },
      {
        "id": "21-1018",
        "name": "Substance Abuse, Behavioral Disorder, and Mental Health Counselors"
      }
    ]
  }
}


In [9]:
# .NET Azure Developer maps to software developer and web developer
# now we need to pick which one of the two would be more appropriate
# the simplest way to do this is using the Job Postings API
payload = {
    "filter": {
        "when": "active",
        "title": ["ET0E8AC162972F4096"]
    },
    "rank": {
        "by": "unique_postings"
    }
}

df = jpa_conn.post_rankings_df(
    "soc5", # the dimension we're ranking by
    payload = payload,  # the query to the API
    querystring = {
        "title_version": "emsi",  # the title version we're using (emsi is the latest version)
        "soc_version": "soc_emsi_2019"   # the occupation version (latest is 2019)
    }  
)

df.head()

,soc5,unique_postings
0,21-1013,112
1,19-3031,9
2,21-1018,6


***
### Summary
As we can see from the example, the top posted SOC for ABA Behavioral Therapist is "21-1013 (Marriage and Family Therapists)". While Emsi generally considers a title to be a combination of job function (SOC) and their skills, this is the simplest way to take a title from Emsi's title library and map it back to a single occupation (SOC) code.